# Cuaderno 02: Entrenamiento de la Vía Auditiva (Fonema -> Clasificación)

**Objetivo:** Entrenar un modelo que pueda identificar un fonema a partir de su representación de audio.

**Flujo de Trabajo:**
1.  **Instalación de Librerías**: Instalar `transformers`, `torch`, `torchaudio` y `librosa`.
2.  **Configuración**: Definir rutas a los datos de audio y para los resultados (embeddings y modelo entrenado).
3.  **Carga de Datos**: Cargar los archivos `.wav` de un idioma (empezaremos con español).
4.  **Extracción de Embeddings con `wav2vec2`**: Procesar cada audio a través de un modelo `wav2vec2` pre-entrenado para extraer representaciones vectoriales (embeddings).
5.  **Preparación del Dataset**: Crear un `Dataset` y `DataLoader` de PyTorch para el entrenamiento.
6.  **Definición y Entrenamiento del Modelo**: Construir y entrenar una CNN 1D para clasificar los embeddings.
7.  **Evaluación y Guardado**: Graficar las métricas de entrenamiento y guardar el modelo final.

## Paso 1: Instalación de Librerías

Instalamos las librerías de `transformers` de Hugging Face (para `wav2vec2`), `pytorch` (para la red neuronal), y `librosa` (una potente herramienta para el análisis de audio).

In [1]:
%pip install transformers torch torchaudio librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda

## Paso 2: Configuración y Rutas

Importamos las librerías necesarias y definimos las rutas a los datos que generamos en el cuaderno anterior. También especificamos el modelo `wav2vec2` que usaremos. `xlsr-53` es un modelo multilingüe excelente.

In [3]:
import torch
import torchaudio
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from pathlib import Path
import numpy as np
import pandas as pd

# --- Selección de Idioma ---
# Cambia a 'en' para procesar el dataset en inglés
LANGUAGE = 'es'

# --- Rutas de Entrada ---
project_root = Path.cwd().parent
audio_dir = project_root / f"data/02_processed/phoneme_audio/{LANGUAGE}"

# --- Rutas de Salida ---
embedding_dir = project_root / f"data/02_processed/wav2vec2_embeddings/{LANGUAGE}"
embedding_dir.mkdir(parents=True, exist_ok=True)
model_dir = project_root / "results/trained_models"
model_dir.mkdir(parents=True, exist_ok=True)

# --- Configuración del Modelo ---
# Usaremos un modelo wav2vec2 pre-entrenado multilingüe de Hugging Face
WAV2VEC2_MODEL_NAME = "facebook/wav2vec2-large-xlsr-53"
# Frecuencia de muestreo a la que el modelo fue entrenado (16kHz)
TARGET_SAMPLE_RATE = 16000

## Paso 3: Cargar los Archivos de Audio

Creamos una lista de todos nuestros archivos `.wav`, cargando la onda de audio y su etiqueta de fonema correspondiente. Nos aseguramos de que todos los audios tengan la frecuencia de muestreo correcta (16kHz).

In [4]:
def load_audio_data(directory, sample_rate):
    """Carga todos los archivos .wav de un directorio."""
    audio_data = []
    filepaths = sorted(list(directory.glob("*.wav")))
    
    for path in filepaths:
        try:
            # Usamos librosa para cargar y remuestrear el audio
            waveform, sr = librosa.load(path, sr=sample_rate)
            
            # La etiqueta es el nombre del archivo sin la extensión
            label = path.stem
            
            audio_data.append({
                "path": path,
                "waveform": waveform,
                "sample_rate": sample_rate,
                "label": label
            })
        except Exception as e:
            print(f"Error cargando el archivo {path}: {e}")
            
    return audio_data

# Cargar los datos del idioma seleccionado
phoneme_data = load_audio_data(audio_dir, TARGET_SAMPLE_RATE)
print(f"Se cargaron {len(phoneme_data)} archivos de audio.")
# Mostramos un ejemplo
if phoneme_data:
    print(f"Ejemplo: {phoneme_data[0]['label']}, Longitud de la onda: {len(phoneme_data[0]['waveform'])}")

Se cargaron 27 archivos de audio.
Ejemplo: a, Longitud de la onda: 10368
